In [1]:
pip install flask flask-cors pandas sentence-transformers scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS  
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/bhava/Downloads/ranchi_tourist_spots_full.csv")
df['combined_text'] = df['Description'] + " " + df['Highlights/Notes'].fillna("")


In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['combined_text'].tolist(), convert_to_numpy=True)


In [5]:
def chatbot(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]

    response_text = "Sure! Here are the best matches for your query:\n\n"
    for idx in top_indices:
        place = df.iloc[idx]['Place Name']
        info = df.iloc[idx]['combined_text']
        response_text += f"• {place}: {info}\n"
    return response_text

In [6]:
def chatbot_conversational(query, top_k=3):
    greetings = ["hi", "hello", "hey"]
    farewells = ["bye", "exit", "quit", "goodbye"]

    if query.lower() in greetings:
        return "Hi there! I can help you find the best tourist spots in Ranchi. Ask me anything!"
    elif query.lower() in farewells:
        return "Goodbye! Hope you enjoy Ranchi :)"
    else:
        return chatbot(query, top_k=top_k)

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route("/")
def home():
    return "Chatbot API is running!" 
    
@app.route("/chat", methods=["POST"])
def chat_api():
    data = request.get_json()
    query = data.get("message", "").strip()

    if not query:
        return jsonify({"error": "No message provided"}), 400

    # Get chatbot response
    response = chatbot_conversational(query)

    # If response is empty or irrelevant, give default
    if not response or response.strip() == "":
        response = "Sorry, I'm here to help with your Jharkhand journey!"

    return jsonify({"reply": response})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.9:5000
Press CTRL+C to quit
